In [1]:
import os
import pandas as pd
import requests
import json
import re
from tqdm import tqdm
import time
import numpy as np
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import datetime as dt
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook
import sqlite3
import sys

## A Set api key

In [2]:
api_file = open('api_key.txt')
api_code = "".join(api_file.readlines())

# 1. City

## 1.A ACS download function

In [3]:
class api_fetch_city:   
    def __init__(self, variable_name, entity, database, year):
        
        self.year = year
        self.base = f'https://api.census.gov/data/{self.year}/acs/'
        self.api_base = self.base+database
        self.api_key = f'key={api_code}'
        self.variable_name = "?&get=NAME,"+variable_name + entity
        
        self.api_final = f'{self.api_base}{self.variable_name}{self.api_key}'
        
    def request(self):
        return requests.get(self.api_final)
    
    def download(self):
        r = requests.get(self.api_final)
        data = r.json()
        df = pd.DataFrame(data)
        df = df.rename(columns=df.iloc[0]).drop(df.index[0])
        df = pd.concat([df.NAME, df.loc[:, ~df.columns.isin(['NAME'])].astype(float)],axis=1)
        return df
  

## 1.1 Download 1 year

In [4]:
city = '&for=place:*&'
df = api_fetch_city('B25008_003E', city, 'acs5', str(2019)).download()
mask = df.state == 6
df = df.loc[mask, :]
df.head(2)

,NAME,B25008_003E,state,place
359,"Moss Landing CDP, California",0.0,6.0,49488.0
360,"Belvedere city, California",517.0,6.0,5164.0


# 2 County
## 2.A ACS download function

In [21]:
class api_fetch:   
    def __init__(self, variable_name, entity, database, year):
        
        self.year = year
        self.base = f'https://api.census.gov/data/{self.year}/acs/'
        self.api_base = self.base+database
        self.api_key = f'key={api_code}'
        self.variable_name = "?&get="+variable_name + entity
        
        self.api_final = f'{self.api_base}{self.variable_name}{self.api_key}'
        
    def request(self):
        return requests.get(self.api_final)
    
    def download(self):
        r = requests.get(self.api_final)
        data = r.json()
        df = pd.DataFrame(data)
        df = df.rename(columns=df.iloc[0]).drop(df.index[0])
        df = df.astype(float)
        return df

## 2.B Set variables for download

In [19]:
codes = {'B25008_003E': 'pop_renter_check',
 'B06011_001E': 'median_inc',
 'B01003_001E': 'pop_total',
 'B01002_001E': 'age',
 'B02001_002E': 'pop_white',
 'B02001_003E': 'pop_black',
 'B25106_028E': 'Burden_30 Less than 20,000',
 'B25106_032E': 'Burden_30 $20,000 to $34,999:',
 'B25106_036E': 'Burden_30 $35,000 to $49,999:',
 'B25106_040E': 'Burden_30 $50,000 to $74,999:',
 'B25106_044E': 'Burden_30 $75,000 or more:',
 'B25111_001E': 'Med_rent'}

## 2.1 Download

In [23]:
county = '&for=county:*&'
df = api_fetch('B25008_003E', county, 'acs5', str(2019)).download()
df.head(3)


,B25008_003E,state,county
1,3623.0,17.0,51.0
2,6196.0,17.0,107.0
3,5537.0,17.0,165.0
